In [85]:
import pandas as pd
import numpy as np

from sklearn.metrics import hamming_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Milestone 3: Traditional statistical and machine learning methods, due Wednesday, April 19, 2017

Think about how you would address the genre prediction problem with traditional statistical or machine learning methods. This includes everything you learned about modeling in this course before the deep learning part. Implement your ideas and compare different classifiers. Report your results and discuss what challenges you faced and how you overcame them. What works and what does not? If there are parts that do not work as expected, make sure to discuss briefly what you think is the cause and how you would address this if you would have more time and resources. 

You do not necessarily need to use the movie posters for this step, but even without a background in computer vision, there are very simple features you can extract from the posters to help guide a traditional machine learning model. Think about the PCA lecture for example, or how to use clustering to extract color information. In addition to considering the movie posters it would be worthwhile to have a look at the metadata that IMDb provides. 

You could use Spark and the [ML library](https://spark.apache.org/docs/latest/ml-features.html#word2vec) to build your model features from the data. This may be especially beneficial if you use additional data, e.g., in text form.

You also need to think about how you are going to evaluate your classifier. Which metrics or scores will you report to show how good the performance is?

The notebook to submit this week should at least include:

- Detailed description and implementation of two different models
- Description of your performance metrics
- Careful performance evaluations for both models
- Visualizations of the metrics for performance evaluation
- Discussion of the differences between the models, their strengths, weaknesses, etc. 
- Discussion of the performances you achieved, and how you might be able to improve them in the future

#### Preliminary Peer Assessment

It is important to provide positive feedback to people who truly worked hard for the good of the team and to also make suggestions to those you perceived not to be working as effectively on team tasks. We ask you to provide an honest assessment of the contributions of the members of your team, including yourself. The feedback you provide should reflect your judgment of each team member’s:

- Preparation – were they prepared during team meetings?
- Contribution – did they contribute productively to the team discussion and work?
- Respect for others’ ideas – did they encourage others to contribute their ideas?
- Flexibility – were they flexible when disagreements occurred?

Your teammate’s assessment of your contributions and the accuracy of your self-assessment will be considered as part of your overall project score.

Preliminary Peer Assessment: [https://goo.gl/forms/WOYC7pwRCSU0yV3l1](https://goo.gl/forms/WOYC7pwRCSU0yV3l1)

## Questions to answer: 

- **What are we predicting exactly?**

So, we are trying to predict movie genres. However, we have that each movie has multiple genres. This leads to the question of how we can predict multiple classifiers for the same object. This more general question is called a multilabel clasification problem. We will explore some of our specifications for this problem below. 

One of the best and most standard solution to do multilable classification is called "one vs. rest" classifiers. These classifiers create n models for each of the n labels. One of the advantages of this model is its interpretability and, for our cases, its ease. We can easily create a pipeline that then does these predictions for us. For an implementation of one vs. all, look at scikit learn: http://scikit-learn.org/dev/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier

We will likely be using this in our early attempts at classification. 

- **What does it means to be succesful? What is our metric for success?**

*adapted from http://people.oregonstate.edu/~sorowerm/pdf/Qual-Multilabel-Shahed-CompleteVersion.pdf*

Here are a few options for our measure of accuracy:

#### Exact Match Ratio
The exact match ratio only considers a correct answer for our multilabel data if it is exactly correct (e.g. if there are three classes, we only classify this as correct if we correctly identify all three classes.) 

#### Accuracy 
Accuracy is a simple way of "goodness of prediction." It is defined as follows 

$$ \frac{1}{n} \sum_i^{n}  \frac{|Y_i\cap Z_i|}{|Y_i \cup Z_i|}$$

Where $$Y_i\cap Z_i $$ refers to the total number of correctly predicted labels over the total number of labels for that instance. So, if for example we predicted [romance, action]  and the true labels were [romance, comedy, horror], this would receive an accuracy of 1/4 because there was one correct prediction and 4 unique labels. 


#### Hamming Loss 
The final and most common form of error for multilable predictions is hamming loss. Hamming loss takes into account both the prediction error (an incorrect error is predicted) and the missing error (a relevant lable is NOT predicted.) this is defined as follows below 

$$ \text{HammingLoss, HL} = \frac{1}{kn} \sum_{i}^{n} \sum_l^k [l \in  Z_i \wedge l \notin Y_i)  + I(l \notin Z_i \wedge  l \in Y_i)]$$

*For this project, we will use the hamming loss, which is defined above.* There is a convenient function in `sklearn` to calculate hamming loss: `sklearn.metrics.hamming_loss`

- What is our first modeling approach? Why? 

- What is our second modeling approach? Why? 

In [15]:
'''
An example of hamming loss. We have true labels:

[0, 1]
[1, 1]

And predicted labels:

[0, 0]
[0, 0]

Hamming loss is .75
'''
hamming_loss(np.array([[0, 1], [1, 1]]), np.zeros((2, 2)))

0.75

### Data Collection & Cleaning

In [49]:
train = pd.read_csv("../data/train.csv")

# drop a rogue column
train.drop("Unnamed: 0", axis = 1, inplace = True)
print "Dataframe shape:", train.shape
train.head(1)

Dataframe shape: (540, 29)


,10402,10749,10751,10752,12,14,16,18,27,28,...,lead actors,movie_id,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,0,0,1,0,0,0,1,0,0,0,...,"[u'Alec Baldwin', u'Miles Bakshi', u'Jimmy Kim...",295693,A story about how a new baby's arrival impacts...,305.881041,/unPB1iyEeTBcKiLg8W083rlViFH.jpg,2017-03-23,The Boss Baby,False,5.7,510


In [50]:
# check for null values
train.isnull().sum()

10402           0
10749           0
10751           0
10752           0
12              0
14              0
16              0
18              0
27              0
28              0
35              0
36              0
37              0
53              0
80              0
878             0
9648            0
adult           0
director        0
lead actors     0
movie_id        0
overview        3
popularity      0
poster_path     0
release_date    0
title           0
video           0
vote_average    0
vote_count      0
dtype: int64

# Model 1: Random Forest

Some thoughts:
    * Random forests don't accept strings, so we'll need to vectorize all of the string variables or exclude them entirely. 

In [81]:
train.columns

Index([u'10402', u'10749', u'10751', u'10752', u'12', u'14', u'16', u'18',
       u'27', u'28', u'35', u'36', u'37', u'53', u'80', u'878', u'9648',
       u'adult', u'director', u'lead actors', u'movie_id', u'overview',
       u'popularity', u'poster_path', u'release_date', u'title', u'video',
       u'vote_average', u'vote_count'],
      dtype='object')

In [118]:
string_matrix["lead actors"][0]

"[u'Alec Baldwin', u'Miles Bakshi', u'Jimmy Kimmel']"

In [93]:
string_cols = ["director", "lead actors", "overview", "title"]
string_matrix = train[string_cols]
map(lambda x: "".join(x), string_matrix["lead actors"])

["[u'Alec Baldwin', u'Miles Bakshi', u'Jimmy Kimmel']",
 "[u'Emma Watson', u'Dan Stevens', u'Luke Evans']",
 "[u'Hugh Jackman', u'Patrick Stewart', u'Dafne Keen']",
 "[u'Matthew McConaughey', u'Reese Witherspoon', u'Seth MacFarlane']",
 "[u'Tom Hiddleston', u'Samuel L. Jackson', u'Brie Larson']",
 "[u'Chris Pratt', u'Bryce Dallas Howard', u'Irrfan Khan']",
 "[u'Scarlett Johansson', u'Pilou Asb\\xe6k', u'Takeshi Kitano']",
 "[u'James McAvoy', u'Anya Taylor-Joy', u'Betty Buckley']",
 "[u'Eddie Redmayne', u'Colin Farrell', u'Katherine Waterston']",
 "[u'Vin Diesel', u'Dwayne Johnson', u'Jason Statham']",
 "[u'Felicity Jones', u'Diego Luna', u'Ben Mendelsohn']",
 "[u'Tom Hardy', u'Charlize Theron', u'Nicholas Hoult']",
 "[u'Amy Adams', u'Jeremy Renner', u'Forest Whitaker']",
 "[u'Matthew McConaughey', u'Jessica Chastain', u'Anne Hathaway']",
 "[u'Chris Pratt', u'Zoe Saldana', u'Dave Bautista']",
 "[u'Keanu Reeves', u'Michael Nyqvist', u'Alfie Allen']",
 "[u'Benedict Cumberbatch', u'Chiwete

In [78]:
labels = train.columns[:17]
features = train.columns[17:]
X = train[features]
y = train[labels]

In [80]:
X

,adult,director,lead actors,movie_id,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,Tom McGrath,"[u'Alec Baldwin', u'Miles Bakshi', u'Jimmy Kim...",295693,A story about how a new baby's arrival impacts...,305.881041,/unPB1iyEeTBcKiLg8W083rlViFH.jpg,2017-03-23,The Boss Baby,False,5.7,510
1,False,Bill Condon,"[u'Emma Watson', u'Dan Stevens', u'Luke Evans']",321612,A live-action adaptation of Disney's version o...,149.542760,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,2017-03-16,Beauty and the Beast,False,6.9,1690
2,False,James Mangold,"[u'Hugh Jackman', u'Patrick Stewart', u'Dafne ...",263115,"In the near future, a weary Logan cares for an...",79.627847,/45Y1G5FEgttPAwjTYic6czC9xCn.jpg,2017-02-28,Logan,False,7.5,2429
3,False,Garth Jennings,"[u'Matthew McConaughey', u'Reese Witherspoon',...",335797,A koala named Buster recruits his best friend ...,77.930498,/s9ye87pvq2IaDvjv9x4IOXVjvA7.jpg,2016-11-23,Sing,False,6.7,1170
4,False,Jordan Vogt-Roberts,"[u'Tom Hiddleston', u'Samuel L. Jackson', u'Br...",293167,Explore the mysterious and dangerous home of t...,61.012215,/5wBbdNb0NdGiZQJYoKHRv6VbiOr.jpg,2017-03-08,Kong: Skull Island,False,6.0,1203
5,False,Colin Trevorrow,"[u'Chris Pratt', u'Bryce Dallas Howard', u'Irr...",135397,Twenty-two years after the events of Jurassic ...,56.445769,/jjBgi2r5cRt36xF6iNUEhzscEcb.jpg,2015-06-09,Jurassic World,False,6.5,6725
6,False,Rupert Sanders,"[u'Scarlett Johansson', u'Pilou Asb\xe6k', u'T...",315837,"In the near future, Major is the first of her ...",54.907680,/myRzRzCxdfUWjkJWgpHHZ1oGkJd.jpg,2017-03-29,Ghost in the Shell,False,6.1,582
7,False,M. Night Shyamalan,"[u'James McAvoy', u'Anya Taylor-Joy', u'Betty ...",381288,Though Kevin has evidenced 23 personalities to...,44.703317,/rXMWOZiCt6eMX22jWuTOSdQ98bY.jpg,2017-01-19,Split,False,6.8,1869
8,False,David Yates,"[u'Eddie Redmayne', u'Colin Farrell', u'Kather...",259316,"In 1926, Newt Scamander arrives at the Magical...",40.852480,/gri0DDxsERr6B2sOR1fGLxLpSLx.jpg,2016-11-16,Fantastic Beasts and Where to Find Them,False,7.0,3305
9,False,F. Gary Gray,"[u'Vin Diesel', u'Dwayne Johnson', u'Jason Sta...",337339,When a mysterious woman seduces Dom into the w...,36.623332,/iNpz2DgTsTMPaDRZq2tnbqjL2vF.jpg,2017-04-12,The Fate of the Furious,False,7.2,466


In [79]:
forest = RandomForestClassifier(n_estimators=100, random_state=109)

# instantiate the classifier (n_jobs = -1 tells it)
# to fit using all CPUs
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

# fit the multi-target random forest
fitted_forest = multi_target_forest.fit(X, y)

# predict the label matrix
preds = fitted_forest.predict(X)

ValueError: could not convert string to float: Going in Style